## Building a Database for crime reports

In this project, we will build a database for storing data related with crimes that occurred in Boston.

We will also create two user groups:

- readonly: Users in this group will have permission to read data only.
- readwrite: Users in this group will have permissions to read and write data but not to delete any data.

### Creating the database and schema

We will start by creating a database named `crime_db` and a schema named `crimes` for storing the tables for the crime data.

In [3]:
import psycopg2
conn = psycopg2.connect(dbname="dq", user="dq")
# set autocommit to True
conn.autocommit = True
cur = conn.cursor()
# create the crime_db database
cur.execute("CREATE DATABASE crime_db;")
conn.close()

In [4]:
# connect to crime_db database
conn = psycopg2.connect(dbname="crime_db", user="dq")
# set autocommit to True
conn.autocommit = True
cur = conn.cursor()
# create the crimes schema
cur.execute("CREATE SCHEMA crimes;")

### Obtaining column names and sample data

Before we start creating tables, let's gather some data about our crime dataset so that can identify the right datatypes to use in our table.

In [5]:
import csv
with open('boston.csv') as file:
    reader = csv.reader(file)
    col_headers = next(reader)
    first_row = next(reader)

### Creating a function for analyzing column values

Let's create a function `get_col_value_set` that given a CSV file and a column index computes a Python set with all distinct values in that column.

In [6]:
def get_col_value_set(csv_filename, col_index):
    values = set()
    with open(csv_filename, 'r') as f:
        next(f)
        reader = csv.reader(f)
        for row in reader:
            values.add(row[col_index])
    return values

for i in range(len(col_headers)):
    values = get_col_value_set("boston.csv", i)
    print(col_headers[i], len(values), sep='\t')

incident_number	298329
offense_code	219
description	239
date	1177
day_of_the_week	7
lat	18177
long	18177


### Finding the maximum length of description column

Let's compute the maximum length of each value in the description column.

In [8]:
descriptions = get_col_value_set("boston.csv", 2) # description is at index number 2
max_len = 0
for description in descriptions:
    max_len = max(max_len, len(description))
print(max_len)

58


### Creating the table

Let's print the value of `col_headers` and `first_row` to identify suitable column names for the table.

In [9]:
print(col_headers)
print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


Below are the datatypes we have chosen for each column:

- incident_number: INTEGER PRIMARY KEY
- offence_code: INTEGER
- description: VARCHAR(100)
- date: DATE
- day_of_the_week: We will create an enumerated datatype named weekday since there there only seven possible values.
- lat: DOUBLE PRECISION
- long: DOUBLE PRECISION

Let's create the table for storing the Boston crime data.

In [10]:
# create the enumerated datatype for the weekday
cur.execute("""
    CREATE TYPE weekday AS ENUM ('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday');
""")
# create the table
cur.execute("""
    CREATE TABLE crimes.boston_crimes (
        incident_number INTEGER PRIMARY KEY,
        offense_code INTEGER,
        description VARCHAR(100),
        date DATE,
        day_of_the_week weekday,
        lat DOUBLE PRECISION,
        long DOUBLE PRECISION
    );
""")

### Loading the data

Now that we have created the table, we can load the data into it.

In [11]:
# load the data from boston.csv into the table boston_crimes in the crimes schema
with open("boston.csv") as f:
    cur.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;", f)

cur.execute("SELECT * FROM crimes.boston_crimes")
# print the number of rows to ensure that they were loaded
print(len(cur.fetchall()))

298329


### Revoking public privileges

Now it is time to handle users. Our goal is to create the two user groups: readonly and readwrite. 

By following the least privilege principle, we will revoke all privileges on schema and database to make sure that there are no privileges inherited from the public group and on the public schema.

In [12]:
cur.execute("REVOKE ALL ON SCHEMA public FROM public;")
cur.execute("REVOKE ALL ON DATABASE crime_db FROM public;")

### Creating user groups

The next step is to create our two users groups.

The readonly group is supposed to only have privileges to perform SELECT queries. In contrast, we want the readwrite group to be able to perform SELECT, INSERT, DELETE and UPDATE queries. Both these uses will also need connection privileges on the crime_db and usage on the crimes schema.

In [13]:
# create groups
cur.execute("CREATE GROUP readonly NOLOGIN;")
cur.execute("CREATE GROUP readwrite NOLOGIN;")
# grant connection to crime_db
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readonly;")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readwrite;")
# grant usage to the crimes schema
cur.execute("GRANT USAGE ON SCHEMA crimes TO readonly;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readwrite;")
# grant group specific privileges to each group
cur.execute("GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;")
cur.execute("GRANT SELECT, INSERT, UPDATE, DELETE ON ALL TABLES IN SCHEMA crimes TO readwrite;")

### Creating users

Let's create a user data_analyst with password secret1 in the readonly group and a user data_scientist with password secret2 in the readwrite group.

In [14]:
# create user data_analyst
cur.execute("CREATE USER data_analyst WITH PASSWORD 'secret1';")
cur.execute("GRANT readonly TO data_analyst;")
# create user data_scientist
cur.execute("CREATE USER data_scientist WITH PASSWORD 'secret2';")
cur.execute("GRANT readwrite TO data_scientist;")

### Testing

Finally, let's test the database setup using SQL queries on the pg_roles table and information_schema.table_privileges.

In the pg_roles table we will check database related privileges in following columns:

- rolname: The name of the user / group that the privilege refers to.
- rolsuper: Whether this user / group is a super user. It should be set to False on every user / group that we have created.
- rolcreaterole: Whether user / group can create users, groups or roles. It should be False on every user / group that we have created.
- rolcreatedb: Whether user / group can create databases. It should be False on every user / group that we have created.
- rolcanlogin: Whether user / group can login. It should be True on the users and False on the groups that we have created.

In the information_schema.table_privileges we will check privileges related to SQL queries on tables.

In [15]:
# check users and groups
cur.execute("""
    SELECT rolname, rolsuper, rolcreaterole, rolcreatedb, rolcanlogin FROM pg_roles
    WHERE rolname IN ('readonly', 'readwrite', 'data_analyst', 'data_scientist');
""")
for user in cur:
    print(user)

# check privileges
cur.execute("""
    SELECT grantee, privilege_type
    FROM information_schema.table_privileges
    WHERE grantee IN ('readonly', 'readwrite');
""")
for user in cur:
    print(user)
    
# close connection
conn.close()

('readonly', False, False, False, False)
('readwrite', False, False, False, False)
('data_analyst', False, False, False, True)
('data_scientist', False, False, False, True)
('readonly', 'SELECT')
('readwrite', 'INSERT')
('readwrite', 'SELECT')
('readwrite', 'UPDATE')
('readwrite', 'DELETE')
